# Combining Naive Bayes Models

Here I will 
1. Combine GaussianNB, MultinomialNB, BernoulliNB by fitting each model with the appropriate data type. 
2. I will try a power transformation to make the data more Gaussian or 
3. I will write a custom NonParametricNB to model using the KernelDensity KernelDensityNB or
4. I will fit an appropriate skewed distribution 

In [1]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [49]:
import pandas as pd
import numpy as np

In [64]:
import sys
sys.path.append("../")

from src.preprocessing import NaNDropper

In [6]:
df = pd.read_csv("../data/raw/train.csv").drop('PassengerId', axis=1)
dfX = df.drop('Survived', axis=1)
dfy = df.Survived

In [68]:
from sklearn.preprocessing import power_transform
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import PowerTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KernelDensity

In [70]:
class KernelDensityNB(BaseEstimator, ClassifierMixin):
    """Bayesian generative classification based on KDE
    
    Parameters
    ----------
    bandwidth : float
        the kernel bandwidth within each class
    kernel : str
        the kernel name, passed to KernelDensity
    """
    def __init__(self, bandwidth=1.0, kernel='gaussian'):
        self.bandwidth = bandwidth
        self.kernel = kernel
        
    def fit(self, X, y):
        self.classes_ = np.sort(np.unique(y))
        training_sets = [X[y == yi] for yi in self.classes_]
        self.models_ = [KernelDensity(bandwidth=self.bandwidth,
                                      kernel=self.kernel).fit(Xi)
                        for Xi in training_sets]
        self.logpriors_ = [np.log(Xi.shape[0] / X.shape[0])
                           for Xi in training_sets]
        return self
        
    def predict_proba(self, X):
        logprobs = np.array([model.score_samples(X)
                             for model in self.models_]).T
        result = np.exp(logprobs + self.logpriors_)
        return result / result.sum(1, keepdims=True)
        
    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), 1)]

In [65]:
pre1 = Pipeline([
    ('nan_drpr', NaNDropper(['Embarked']))
])

dfX, dfy = pre1.fit_transform(dfX, dfy)

## Preprocessing

In [69]:
pre2 = Pipeline([
    ('imp', SimpleImputer()),
    ('powtrans', PowerTransformer(method='box-cox'))
], 'passthrough')

precomb = ColumnTransformer([
    ('clmn_drpr', 'drop', ['Name', 'Ticket', 'Cabin']),
    ('enc', OneHotEncoder(drop='first'), ['Sex', 'Embarked']),
    ('imp_powtrans', pre2, ['Age'])
], 'passthrough')

I will 
1. Use the boxcox transformation for Age and model GaussianNB on it
2. Fit KernelDensityNB on Fare 
3. MultinomialNB on Parch, and SibSp
4. BernoulliNB on Sex and Embarked after OneHotEncoding

In [ ]:
GaussianNB()

In [ ]:
MultinomialNB()

In [ ]:
BernoulliNB()

In [ ]:
KernelDensityNB()

In [ ]:
class HybridNB(BaseEstimator, ClassifierMixin):
    
    def __init__(self, 
                gcols, mcols, bcols, kcols,
                gpriors=None, gvar_smoothing=1e-09,
                mfit_prior=True, mclass_prior=None, 
                balpha=1.0, bbinarize=0.0, bfit_prior=True, bclass_prior=None,
                kbandwidth=1.0, kkernel='gaussian'):
        
        self.gcols, self.mcols, self.bcols, self.kcols = gcols, mcols, bcols, kcols
        self.gpriors, self.gvar_smoothing = gpriors, gvar_smoothing
        self.mfit_prior, self.mclass_prior = mfit_prior, mclass_prior
        self.balpha, self.bbinarize, self.bfit_prior, self.bclass_prior